# Proof of concept example (ECAS-B2SHARE)

## 1. Use Case 

<img src="ecas-b2share.png" width="700px" alt="Jupyter Notebook example"/>

### 2. Calculate number of tropical nights with Ophidia

In [ ]:
from PyOphidia import cube
cube.Cube.setclient(read_env=True)

In [ ]:
mintemp = cube.Cube(src_path='/data-training/E63_kp40_198001_199012_T2m_daymin_merged.nc',
    measure='temp2',
    import_metadata='yes',
    imp_dim='time',
    imp_concept_level='d', vocabulary='CF',hierarchy='oph_base|oph_base|oph_time',
    ncores=4,
    description='Min Temps'
    )

In [ ]:
tropicalnights = mintemp.apply(
    query="oph_predicate('OPH_FLOAT','OPH_INT',measure,'x-293.15','>0','1','0')"
)

In [ ]:
count = tropicalnights.reduce2(
    operation='sum',
    dim='time',
    concept_level='y',
)

Export result to netCDF 

In [ ]:
count.exportnc2(output_path='/data/ecas/b2share', output_name='', export_metadata='no')

In [ ]:
firstyear = count.subset(subset_filter=1, subset_dims='time')

Plot the result 

In [ ]:
%matplotlib inline
data = firstyear.export_array(show_time='yes')
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, cm
import numpy as np

lat = data['dimension'][0]['values'][:]
lon = data['dimension'][1]['values'][:]
var = data['measure'][0]['values'][:]
var = np.reshape(var, (len(lat), len(lon)))

fig = plt.figure(figsize=(15, 15), dpi=100)
ax  = fig.add_axes([0.1,0.1,0.8,0.8])

map = Basemap(projection='cyl',llcrnrlat= -90,urcrnrlat= 90, llcrnrlon=0,urcrnrlon=360, resolution='c')

map.drawcoastlines()
map.drawparallels(np.arange( -90, 90,30),labels=[1,0,0,0])
map.drawmeridians(np.arange(-180,180,30),labels=[0,0,0,1])

x, y = map(*np.meshgrid(lon,lat))

clevs = np.arange(0,371,10)

cnplot = map.contourf(x,y,var,clevs,cmap=plt.cm.jet)
cbar = map.colorbar(cnplot,location='right')

plt.title('Tropical Nights (year 1980)')
plt.savefig('tn.png')
plt.show()

### 3. Create draft record in B2SHARE and add file (exported netCDF)

In [ ]:
from ecasb2share.ecasb2shareclient import EcasShare as Client
client = Client()

In [ ]:
client.list_communities()

In [ ]:
title = 'Tropical Nights'
community_id = 'e9b9792e-79fb-4b07-b6b4-b9c2bd06d095'
file = 'b2share/temp2.nc'
filebucketid = 'c7a3c9db-803e-425c-8d20-3c521df08b56'

In [ ]:
client.create_draft_record('e9b9792e-79fb-4b07-b6b4-b9c2bd06d095', 'test')

In [ ]:
client.add_file_to_draft_record(file, filebucketid)

Clean workspace

In [ ]:
cube.Cube.client.submit("oph_delete cube=[container=E63_kp40_198001_199012_T2m_daymin_merged.nc]")
cube.Cube.deletecontainer(delete_type="physical", hidden="no", container='E63_kp40_198001_199012_T2m_daymin_merged.nc')

In [ ]:
client.delete_draft_record('50a62935bce44199bac4436f90c3ac3c')